In [ ]:
import m3u8
import requests
import os
from concurrent.futures import ThreadPoolExecutor

In [ ]:
class StreamDownloader:
    def __init__(self, maxThreads, segments, streamType):
        self.maxThreads = maxThreads
        self.streamType = streamType
        self.segments = segments

    def getProperties(self):
        duration = sum([float(segment.duration) for segment in self.segments])
        duration = f'{int(duration) // 3600}:{int(duration) // 60 % 60}:{int(duration) % 60}'
        return (duration, len(self.segments))

    def download(self, segmentFile, segmentUrl):
        if (os.path.exists(segmentFile)):
            print(f'Exist : {segmentFile}')
            return
        response = requests.get(segmentUrl, stream=True)
        if response.status_code == 200:
            with open(segmentFile + '.temp', 'wb') as f:
                for chunk in response.iter_content(chunk_size=102400):
                    f.write(chunk)
            os.rename(segmentFile + '.temp', segmentFile)
            print(f"Downloaded: {segmentFile}")
        else:
            print(f"Failed to download segment: {segmentUrl}")


    def downloadSegments(self):
        with ThreadPoolExecutor(max_workers=self.maxThreads,) as executor:
            futures = []
            for index, segment in enumerate(self.segments):
                futures.append(executor.submit(self.download, f'{self.streamType}_{index}.ts', segment.uri))
            if(input('Press any key to stop : ')):
                executor.shutdown(wait=True, cancel_futures=True)


    def mergeTs(self):
        exportFile = f'{self.streamType}.ts'
        with open(exportFile, 'wb') as outfile:
            for index, segment in enumerate(self.segments):
                with open(f'{self.streamType}_{index}.ts', 'rb') as input:
                    outfile.write(input.read())
        print(f'Final {self.streamType} file created : {exportFile}')



In [ ]:
audioSegments = m3u8.load('au.txt').segments
aSt = StreamDownloader(maxThreads=25, segments=audioSegments, streamType='audio')
aSt.getProperties()


In [ ]:
videoSegments = m3u8.load('v.txt').segments
vSt = StreamDownloader(maxThreads=25, segments=videoSegments, streamType='video')
vSt.getProperties()